In [ ]:
// #Copyright 2020 Vraj Shah, Arun Kumar
// #
// #Licensed under the Apache License, Version 2.0 (the "License");
// #you may not use this file except in compliance with the License.
// #You may obtain a copy of the License at
// #
// #    http://www.apache.org/licenses/LICENSE-2.0
// #
// #Unless required by applicable law or agreed to in writing, software
// #distributed under the License is distributed on an "AS IS" BASIS,
// #WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// #See the License for the specific language governing permissions and
// #limitations under the License.

%classpath add mvn com.salesforce.transmogrifai transmogrifai-core_2.11 0.7.0

In [ ]:
%classpath add mvn org.apache.spark spark-mllib_2.11 2.4.5

In [ ]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf

import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.evaluators.Evaluators
import java.io._


In [ ]:
import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders

In [ ]:
// package com.salesforce.op.readers

import com.salesforce.op.test.PassengerSparkFixtureTest
import org.apache.avro.Schema
import org.apache.avro.generic.GenericRecord
import org.junit.runner.RunWith
import org.apache.avro.generic.GenericRecord
// import org.scalatest.FlatSpec
// import org.scalatest.junit.JUnitRunner
import scala.collection.JavaConverters._
import java.nio.file.Paths
import java.io.File
import org.apache.spark.sql.execution.datasources.csv.CSVSchemaUtils
import com.salesforce.op.utils.io.csv.{CSVInOut, CSVOptions, CSVToAvro}

In [ ]:
val conf = new SparkConf().setMaster("local[*]").setAppName("TestPrediction")

// conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer") 
// conf.registerKryoClasses(Array(classOf[org.apache.avro.generic.GenericData$Record]))

conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.registerKryoClasses(Array(classOf[org.apache.avro.generic.GenericData.Record]))

implicit val spark = SparkSession.builder.config(conf).getOrCreate()

In [ ]:
def testDataDir: String = {
Some(new File("test_csv"))
  .collect{ case d if d.isDirectory => d.getPath}
  .getOrElse(Paths.get("test_csv-sibling").relativize(Paths.get("test_csv")).toString)
}

In [ ]:
val fw = new FileWriter("TransmogrifAI_Predictions.txt", true)

for (i <- 0 until 1985) {
print(i)


val istr = i.toString
// var Newstr = '../../' + '/' + istr + ".csv"    
var Newstr = testDataDir + '/' + istr + ".csv"
// def passengerCsvWithHeaderPath: String = Paths.get(Newstr, "2.csv").toString

val finalPath = Newstr
val options: CSVOptions = CSVDefaults.CSVOptions
val csvData = new CSVInOut(options).readRDD(finalPath)
val headers: Seq[String] = Seq.empty
val hdrs = if (headers.nonEmpty) headers else csvData.first()
val hdrsSet = hdrs.toSet
val data = csvData.filter(_.exists(!hdrsSet.contains(_)))

val columnPrunning = spark.sessionState.conf.csvColumnPruning

val inferredSchema = CSVSchemaUtils.infer(data.map(_.toArray), hdrs, options, columnPrunning)

fw.write(inferredSchema.toString)
fw.write("\n")
    
println()
println(inferredSchema)
}

fw.close()


In [ ]:
// Python Snippet. So, change kernel to Python 3
// inf_types = pd.read_csv('TransmogrifAI_Predictions.csv', header=None)
// dict_label = {
//     'IntegerType': 0,
//     'DecimalType(4': 0,
//     'DoubleType':0, 
//     'LongType':0, 
//     'DecimalType(3': 0,
//     'StringType': 8,    
//     'TimestampType': 2,
//     'BooleanType': 8  
// }

// inf_types[1] = [dict_label[i] for i in inf_types[1]]
// y_trans = inf_types[1].values.tolist()

// df = pd.read_csv('../data_test.csv')
// y_true = df.y_act.values.tolist()
                 
// print(accuracy_score(y_true, y_trans))
// print(confusion_matrix(y_true, y_trans))                 